In [1]:
# general python
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
import numpy as np
import os
from pathlib import Path
import yaml
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import scipy
import xarray as xr
from tqdm import tqdm
import glob
# from devtools import pprint
from rich import print
from tqdm import tqdm

In [2]:
# general eWC
import ewatercycle
import ewatercycle.forcing
import ewatercycle.models

In [3]:
path = Path.cwd()
forcing_path = path / "Forcing"
observations_path = path / "Observations"
figure_path = path / "Figures"
output_path = path / "Output"
forcing_path

PosixPath('/home/davidhaasnoot/eWaterCycle-WSL-WIP/Forcing')

Simple example using HBV:

In [4]:
from ewatercycle.forcing import sources

In [5]:
import importlib.util

def module_from_file(module_name, file_path):
    spec = importlib.util.spec_from_file_location(module_name, file_path)
    module = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(module)
    return module

In [6]:
DA = module_from_file("DA",r'../eWaterCycle-DA/src/ewatercycle_DA/DA.py')

In [7]:
n_particles = 10

In [8]:
ensemble = DA.Ensemble(N=n_particles)
ensemble.setup()

In [9]:
experiment_start_date = "1997-08-01T00:00:00Z"
experiment_end_date = "1999-03-01T00:00:00Z"
HRU_id = 14138900

In [10]:
forcing_args = []
for i in range(1,11):
    forcing_args.append(dict(
        start_time = experiment_start_date,
        end_time = experiment_end_date,
        directory = forcing_path,
        camels_file = f'0{HRU_id}_lump_cida_forcing_leap.txt', 
        alpha = 1.23 + (i/100)
    ))

In [11]:
lst_forcing = ensemble.generate_forcing([ewatercycle.forcing.sources.HBVForcing]*10, forcing_args)

In [12]:
lst_forcing

[HBVForcing(start_time='1997-08-01T00:00:00Z', end_time='1999-03-01T00:00:00Z', directory=PosixPath('/home/davidhaasnoot/eWaterCycle-WSL-WIP/Forcing'), shape=None, filenames={}, camels_file='014138900_lump_cida_forcing_leap.txt', pr='.nc', evspsblpot='.nc', tas='.nc', alpha=1.24, test_data_bool=False),
 HBVForcing(start_time='1997-08-01T00:00:00Z', end_time='1999-03-01T00:00:00Z', directory=PosixPath('/home/davidhaasnoot/eWaterCycle-WSL-WIP/Forcing'), shape=None, filenames={}, camels_file='014138900_lump_cida_forcing_leap.txt', pr='.nc', evspsblpot='.nc', tas='.nc', alpha=1.25, test_data_bool=False),
 HBVForcing(start_time='1997-08-01T00:00:00Z', end_time='1999-03-01T00:00:00Z', directory=PosixPath('/home/davidhaasnoot/eWaterCycle-WSL-WIP/Forcing'), shape=None, filenames={}, camels_file='014138900_lump_cida_forcing_leap.txt', pr='.nc', evspsblpot='.nc', tas='.nc', alpha=1.26, test_data_bool=False),
 HBVForcing(start_time='1997-08-01T00:00:00Z', end_time='1999-03-01T00:00:00Z', director

## Using ERA5 data

In [13]:
#shapefile that describes the basin we want to study.
path = Path.cwd()
forcing_path = path / "Forcing"
shapeFile = forcing_path/ "Rhine_shp" / "Rhine.shp"

#location to saved forcing results from previous notebook
forcingLocation = forcing_path / "RhineForcing2000-2002"

#GRDC station ID for the observation station
grdc_station_id = "6335020"  # GRDC station ID
basin_name = "Rhine"
observation_path = path / "Observations"

#period of interest. Make sure that GRDC data is available for this period and that.
experiment_start_time="2000-01-01T00:00:00Z"

In [14]:
ewatercycle.forcing.sources["LumpedMakkinkForcing"]

ewatercycle._forcings.makkink.LumpedMakkinkForcing

In [15]:
n_particles = 6

In [16]:
ensemble = DA.Ensemble(N=n_particles)
ensemble.setup()

In [17]:
forcing_args = []
for i in range(1,n_particles+1):
    forcing_args.append(dict(
        dataset="ERA5",
        start_time = experiment_start_date,
        end_time = f"2000-0{i}-01T00:00:00Z",
        shape = shapeFile.absolute()
    ))

In [18]:
ewatercycle.forcing.sources["LumpedMakkinkForcing"].generate(**forcing_args[0])

ERROR:esmvalcore._recipe.check:No input files found for Dataset:
{'diagnostic': 'diagnostic',
 'variable_group': 'pr',
 'dataset': 'ERA5',
 'project': 'OBS6',
 'mip': 'day',
 'short_name': 'pr',
 'alias': 'ERA5',
 'frequency': 'day',
 'long_name': 'Precipitation',
 'modeling_realm': ['atmos'],
 'original_short_name': 'pr',
 'preprocessor': 'pr',
 'recipe_dataset_index': 0,
 'standard_name': 'precipitation_flux',
 'tier': 3,
 'timerange': '1997/2000',
 'type': 'reanaly',
 'units': 'kg m-2 s-1',
 'version': 1}
session: 'ewcrep0sjohs3__20240422_144817'
ERROR:esmvalcore._recipe.check:Looked for files matching: /home/davidhaasnoot/dcache/climate-data/obs/Tier3/ERA5/OBS6_ERA5_reanaly_1_day_pr[_.]*nc
ERROR:esmvalcore._recipe.check:Set 'log_level' to 'debug' to get more information
ERROR:esmvalcore._recipe.check:No input files found for Dataset:
{'diagnostic': 'diagnostic',
 'variable_group': 'tas',
 'dataset': 'ERA5',
 'project': 'OBS6',
 'mip': 'day',
 'short_name': 'tas',
 'alias': 'ERA5',


RecipeError: Could not create all tasks

In [ ]:
lst_forcing = ensemble.generate_forcing(ewatercycle.forcing.sources["LumpedMakkinkForcing"].generate, forcing_args)